In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import pandas as pd
import numpy as np
from pathlib import Path


from matplotlib import pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [3]:
## Предоброботка данных


In [4]:
path_work_file = Path('/home/owl/PycharmProjects/Questions_search_task/dataset_of_questions/root_dataset.csv')  # /'root_dataset.txt'

data_sentenes = pd.read_csv(path_work_file, sep='\t', names=['Line informatation', 'Sign'])

In [5]:
data_sentenes['Sign'].size

11214821

In [6]:
data_sentenes.tail(10)

,Line informatation,Sign
11214811,Это излучение и фиксируют камеры.Сами органиче...,.
11214812,Они реагируют на локальную концентрацию кислор...,.
11214813,Сегодня метод люминисцентных преобразователей ...,.
11214814,После отработки исследовательского метода разр...,.
11214815,Так называют один из видов автоколебаний конст...,.
11214816,Хотя в целом причины возникновения бафтинга из...,.
11214817,Этот метод не дает высокого разрешения.Помимо ...,.
11214818,В случае с шелковинками на поверхности модели ...,.
11214819,По их колебанию и судят об аэродинамической фо...,.
11214820,"В случае с цветной масляной пленкой , некоторы...",Сычёв


In [7]:
# from tqdm import tqdm 
signs = ['?', '.', ';', '!', ',', '...', ':']
data_sentenes = data_sentenes.loc[data_sentenes['Sign'].isin(signs)]  # Удаление всех строк, если содержание столбца "Sign" нет в списке допустимых символов |signs: List |

In [8]:
data_sentenes['Sign'].size

9145180

In [9]:
count_trash = 0
all_count = 0
for sign in data_sentenes['Sign']:
    if sign not in signs:
        count_trash += 1
    else:
        all_count += 1
print(f'Предложений с мусором: {count_trash}')
print(f'Всего предложений:     {all_count}')

Предложений с мусором: 0
Всего предложений:     9145180


### Удаление пропущеных значений в строках

In [10]:
data_sentenes['Line informatation'].isnull().sum()

43123

In [11]:
data_sentenes = data_sentenes.dropna()

In [12]:
data_sentenes['Line informatation'].isnull().sum()

0

### Предобработка

In [13]:
sign_dict = {'end_sent_count': {'?': 0, '.': 0, ';': 0, '!': 0, ',': 0, '...': 0, ':': 0}}

count_error = 0
for sign in data_sentenes['Sign']:
    if sign in signs:
        sign_dict['end_sent_count'][sign] += 1
    else:
        count_error +=1
        
print(sign_dict)
print(f'Предложение с мусором: {count_error}')

{'end_sent_count': {'?': 531872, '.': 7810430, ';': 33169, '!': 538016, ',': 30556, '...': 112030, ':': 45984}}
Предложение с мусором: 0


In [14]:
data_sentenes['Sign'].size

9102057

In [15]:
data_sentenes.tail(10)

,Line informatation,Sign
11214810,В ответ на облучение краска начинает испускать...,.
11214811,Это излучение и фиксируют камеры.Сами органиче...,.
11214812,Они реагируют на локальную концентрацию кислор...,.
11214813,Сегодня метод люминисцентных преобразователей ...,.
11214814,После отработки исследовательского метода разр...,.
11214815,Так называют один из видов автоколебаний конст...,.
11214816,Хотя в целом причины возникновения бафтинга из...,.
11214817,Этот метод не дает высокого разрешения.Помимо ...,.
11214818,В случае с шелковинками на поверхности модели ...,.
11214819,По их колебанию и судят об аэродинамической фо...,.


## ---------------------------------------------------------------------------------------------------------------------------------------------------------

Варианты удаления строки из dataframe:
    1) фильтруем по одному значению: df = df.loc[df['STP'] != 1005092]
    2) можно указать несколько значений: df = df.loc[~df['STP'].isin([1005092])]
    Вариант 3: фильтруем по одному значению: df = df.query("STP != 1005092")
    Вариант 4: можно указать несколько значений: df = df.query("STP not in [1005092, ...]")
    Вариант 5: df = df.drop(np.where(df['STP'] == 1005092)[0])

In [16]:
# from tqdm import tqdm 

# signs = ['?', '.', ';', '!', ',', '...', ':']

# for sign in tqdm(enumerate(data_sentenes['Sign']), total=11214821):  # tqdm(enumerate(read_file), total=11214821)
#     if sign not in signs:
#         data_sentenes.drop(index=data_sentenes.index, inplace=True)

## ---------------------------------------------------------------------------------------------------------------------------------------------------------

# Балансировка классов

Так как у нас задача бинарной классификации, а именно: важно понять предложение вопросительный характер - "1" или нет - "0"(То есть все прочие знаки). То Появляется необходимость в соблюдении баланса классво.
Если говорить конкретнее: 
                                     "?": 541 126 обьектов КЛАССА - "1". 
                         Столько же обьектов сделаем и для КЛАССА - "0".
Для упрощения сделаем 540 000 обьектов для классов "1", "0"


При этом важно сохранить пропорцию объектов из класса "0". Так как в реальных примерах для модели вероятность встретить предложение определенного характера будет совпадать с нашими данным - исходя их статистики.



Для этого посчитаем процентное соотношение обьектов различных знаков по отношению ко всем предложениям за вычетом вопросительных, их в отдельную выборку.
после чего через train.test.split поставим необходимое соотношение в оставшейся выборке и соединим её с уже имеющейся выборкой вопросительных знаков

In [17]:
data_of_question = {'Line informatation': [], 'Sign': []}
data_of_question = pd.DataFrame(data_of_question)

In [18]:
data_of_question.head()

,Line informatation,Sign


In [19]:
data_sentenes['Sign'].size

9102057

In [20]:
data_of_question = data_sentenes.loc[data_sentenes['Sign'] == '?']  # Создаем датасет содержащий только вопросы

In [21]:
data_sentenes = data_sentenes.loc[data_sentenes['Sign'] != '?']  # Избавляемся в основном датасете предложений от предложений с вопросом

Теперь мы имеем два датасета - **data_of_question** (что содержит только вопросы. Он же датасет класса 1 )
и **data_sentenes** (что содержит все предложения кроме вопросительных. Он же датасет класса 0)

In [22]:
print(data_sentenes['Sign'].size)
print(data_of_question['Sign'].size)

8570185
531872


In [23]:
data_of_question.head(10)

,Line informatation,Sign
391,Остается главный вопрос – дойдет ли дело до ро...,?
721,Вопрос - куда,?
742,« Высотный регламент » или творчество,?
743,Оживленная дискуссия о современном облике дело...,?
744,"Хотим ли мы , чтобы потомки , рассматривая дос...",?
745,"И какую роль мы , питерцы , для себя отводим :...",?
783,"То же самое касается живописи , театра , кинем...",?
784,И абсолютно дискуссионным является вопрос о то...,?
787,К чему это приведет,?
1225,Но разве можно в марокканской пустыне всем под...,?


In [24]:
data_of_question.index = np.arange(len(data_of_question))  #  даем новые, корректные индексы

In [25]:
data_of_question.head(10)  # проверяем.

,Line informatation,Sign
0,Остается главный вопрос – дойдет ли дело до ро...,?
1,Вопрос - куда,?
2,« Высотный регламент » или творчество,?
3,Оживленная дискуссия о современном облике дело...,?
4,"Хотим ли мы , чтобы потомки , рассматривая дос...",?
5,"И какую роль мы , питерцы , для себя отводим :...",?
6,"То же самое касается живописи , театра , кинем...",?
7,И абсолютно дискуссионным является вопрос о то...,?
8,К чему это приведет,?
9,Но разве можно в марокканской пустыне всем под...,?


In [26]:
# Проверяем, остались ли вопросы в основном датасете "data_sentenes"
print('True') if "?" in data_sentenes['Sign'] else print('False')

False


## Непосредственно создаем единый датасет содержащий одинаковое количество объектов классов с пропорциональной действительность.

Пробую train.test.split для балансировки - *Ответвление* от тематики.

In [27]:
from sklearn.model_selection import train_test_split
percent_qestion_of_sent = round(data_of_question['Sign'].size / data_sentenes['Sign'].size, 4)

print(percent_qestion_of_sent)
train_sent_data, test_sent_data = train_test_split(data_sentenes, test_size=percent_qestion_of_sent, stratify=data_sentenes.Sign)

0.0621


In [28]:
print(test_sent_data.shape)

(532209, 2)


In [29]:
work_dataset = pd.concat([test_sent_data, data_of_question], axis=0)

In [30]:
work_dataset.shape

(1064081, 2)

In [31]:
from sklearn.utils import shuffle


work_dataset = shuffle(work_dataset)
work_dataset.head(10)

,Line informatation,Sign
45254,Путин : Там кухня,?
363593,"Интересно , там сильно переживают насчет & quo...",?
191691,"Что Вы предлагаете делать с самообороной , спо...",?
76269,А еще вопросы о заработках других депутатов за...,?
235698,"Madame , почему такое возбуждение",?
368134,Не верите,?
10734735,А имеющиеся у них `` права '' достаточно специ...,.
488445,"[ id250060519|Vatnik ] , вопрос на вопрос",?
8731640,Мало спал,.
8236241,Все перерисовали в 1920 на 1280 или это вектор...,.


In [32]:
work_dataset.index = np.arange(len(work_dataset))

In [33]:
work_dataset.head(10)

,Line informatation,Sign
0,Путин : Там кухня,?
1,"Интересно , там сильно переживают насчет & quo...",?
2,"Что Вы предлагаете делать с самообороной , спо...",?
3,А еще вопросы о заработках других депутатов за...,?
4,"Madame , почему такое возбуждение",?
5,Не верите,?
6,А имеющиеся у них `` права '' достаточно специ...,.
7,"[ id250060519|Vatnik ] , вопрос на вопрос",?
8,Мало спал,.
9,Все перерисовали в 1920 на 1280 или это вектор...,.


In [34]:
work_dataset.to_csv('/home/owl/PycharmProjects/Questions_search_task/dataset_of_questions/work_dataset.csv', sep='\t')

In [35]:
work_dataset['Sign'].value_counts(normalize=True)

?      0.499842
.      0.455819
!      0.031399
...    0.006538
:      0.002684
;      0.001936
,      0.001783
Name: Sign, dtype: float64

## ---------------------------------------------------------------------------------------------------------------------------------------------------------

## **Следующий раздел посвещен векторизации рабочего датасета. Что бы в дальнейшем подавать его на обучение различным моделям.**

**pipline** для векторизации **TfidfVectorizer**.



Для того что бы получить из предложения вектор.
- Важно для начала понять как работает **TfidfVectorizer**. Из чего он состоит, как рассчитывается.
- После чего мы должны обучить его. через методо **.fit()** при этом на **fit** мы подаем столбец строк из датасета.
- После того как vectorizer обучился. Мы можем получить из него вектора предложений. Но при этом важно понимать что результаты выполненной работы нигде не сохранятся. Плюс ко всему нужно понимать что обрабатывать строки он будет по бачам. 
- Исходя из вышестоящего пункта появляется необходимость в познании того, как сохранить vectorizer - как записать его в бинарном формате, тоесть *сериализовать* его.
- Соответственно после того как мы сохранили vectorizer, нужно знать, как получить информацию зашитую в неём и хранящую результат векторизации строк датасета - *десирилизовать* файл.


После чего уже можно будет прописать обучения моделей.

In [37]:
# from sklearn.feature_extraction.text import TfidfVectorizer

